# Using GPT Function Calling to get deterministic output

## Navigating

In [1]:
pwd

'/home/pranshumaan'

In [2]:
cd /media/pranshumaan/TOSHIBA\ EXT

/media/pranshumaan/TOSHIBA EXT


In [3]:
cd Dev/

/media/pranshumaan/TOSHIBA EXT/Dev


In [4]:
ls

 data/                                  'Project_Finmin (copy)'/
 Deep_Learning_Pytorch_Udemy_Tutorial/   Secrets/
 Langchain/                              TFR_Modelling/
 Project_Finmin/


In [5]:
mkdir GPT_Functin_Calling

In [6]:
cd GPT_Functin_Calling/

/media/pranshumaan/TOSHIBA EXT/Dev/GPT_Functin_Calling


In [7]:
pwd

'/media/pranshumaan/TOSHIBA EXT/Dev/GPT_Function_Calling'

## Passing secrets and loading requirements

In [8]:
import openai
import json

with open('../Secrets/open_ai_api_key.txt', 'r') as file:
    key = file.read().rstrip()

# load and set our key
openai.api_key = key

## Example demo: Without function calling

In [10]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
)

In [11]:
reply_content = completion.choices[0].message.content
print(reply_content)

I am sorry, as an AI language model, I do not have real-time information. Can you please specify the date or time frame you are referring to?


As expected, the response is not useful due to inability of GPT to provide information that can be LLM generated.

## Demonstrating Function Calling

### Using the documentation example

In [39]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

In [40]:
message = response["choices"][0]["message"]
message

<OpenAIObject at 0x7f3c2f058ef0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

In [41]:
if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

In [51]:
function_name, function_args

('get_current_weather', {'location': 'Boston, MA'})

In [52]:
second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

In [53]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7Ry0DnB6HkoAQb8UR84jNWcdcJcGf at 0x7f3c3071bba0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The current weather in Boston is sunny and windy with a temperature of 72 degrees.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686899861,
  "id": "chatcmpl-7Ry0DnB6HkoAQb8UR84jNWcdcJcGf",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 73,
    "total_tokens": 90
  }
}

## Testing with a new example
Getting a stock price from the company name

In [60]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the last price of Airtel's stock?"}],
        functions=[
            {
                "name": "get_closing_stock_price",
                "description": "Get the closing stock price for a given company",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "company_name": {
                            "type": "string",
                            "description": "The company name e.g. Microsoft, Apple etc.",
                        },
                    },
                    "required": ["company_name"],
                },
            }
        ],
        function_call="auto",
    )

In [61]:
message = response["choices"][0]["message"]
message

<OpenAIObject at 0x7f3c2f24db20> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"company_name\": \"Airtel\"\n}",
    "name": "get_closing_stock_price"
  },
  "role": "assistant"
}

In [62]:
if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

In [63]:
function_name, function_args

('get_closing_stock_price', {'company_name': 'Airtel'})

In [64]:
second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What's the price of Airtel's stock today??"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

In [65]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7Ry5h0BGgcjzjVLVOOj5ktDjiMQlt at 0x7f3c2f5d8ae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I apologize, but I don't have access to live stock prices. I recommend checking a financial website or contacting a stockbroker for the latest information on Airtel's stock price.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686900201,
  "id": "chatcmpl-7Ry5h0BGgcjzjVLVOOj5ktDjiMQlt",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 37,
    "prompt_tokens": 81,
    "total_tokens": 118
  }
}

## Testing with another example
Getting the latitude and longitude of a place

In [69]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What are the coordinates of Bengaluru?"}],
        functions=[
            {
                "name": "get_latitude_and_longitude",
                "description": "Get the coordinates in terms of latitude and longitude for a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["degrees"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

In [70]:
message = response["choices"][0]["message"]
message

<OpenAIObject at 0x7f3c2f517dd0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Bengaluru\"\n}",
    "name": "get_latitude_and_longitude"
  },
  "role": "assistant"
}

In [71]:
if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

In [72]:
second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What are the coordinates of Bengaluru?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

In [73]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7Ry9eJnZH16gzUBM0j5ZaZFgITaAk at 0x7f3c2f24d170> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The coordinates of Bengaluru are approximately 12.9716\u00b0 N latitude and 77.5946\u00b0 E longitude.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686900446,
  "id": "chatcmpl-7Ry9eJnZH16gzUBM0j5ZaZFgITaAk",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 75,
    "total_tokens": 100
  }
}

Seems correct, from Google: Bengaluru/Coordinates
12.9716° N, 77.5946° E

In [74]:
third_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What are the coordinates of Timbuktu?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

In [75]:
third_response

<OpenAIObject chat.completion id=chatcmpl-7RyApmm8mfVWbGkgCBvEOMP7WI3D2 at 0x7f3c2f5176f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The coordinates of Timbuktu are approximately 16.7666\u00b0 N latitude and 3.0026\u00b0 W longitude.",
        "role": "assistant"
      }
    }
  ],
  "created": 1686900519,
  "id": "chatcmpl-7RyApmm8mfVWbGkgCBvEOMP7WI3D2",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 77,
    "total_tokens": 104
  }
}

Timbuktu/Coordinates
16.7666° N, 3.0026° W

For comparison, Chat GPT-3.5 answer was: https://chat.openai.com/share/fc17a454-6569-4784-8d19-fffe00ffa5c8
>The coordinates of Timbuktu are approximately 16.7666° N latitude and 3.0026° W longitude. Please note that these coordinates are approximate and should be verified using reliable sources before any specific use.